# **Numerical Analysis Project - Week 3**
*Mathilde Bénard, Hector Giraud, Emilie Soret - TEAM 5, GROUP E*

---

In [16]:
import numpy as np
from scipy.integrate import solve_ivp

For our team : E.5, we have to consider the following problem 

$$ \begin{cases}
\partial_t u = \beta \partial^2_x u \\
u(t, x=0)=a \\
u(t=0,x)=u_1 cos(\frac{\pi x}{2L}) \\
\partial_x(t,x=L)=0
\end{cases}$$

$$\forall J \in \N, \text{we define } \\
h=\frac{L}{J+1} \\
\forall j \in [1,J], x_j=jh $$

1. This is a PDE that is homogeneous and autonome as f=0. It is also linear. 

2. Let 
$$J \in \N \\
\text{We define} \\
h = \frac{L}{J+1} \\
\forall j \in \llbracket 1, J \rrbracket, x_j = jh \\$$

To implement the scheme we do the following approximations : 

(Forward difference at L): We need to define an extra point for u at x=L+h. We define it s.t. 
 $$\partial_x(t,x=L) \approx \frac{u(t,x=L+h)-u(t,x=L)}{h}=0 \\ \text{which implies that} \\
 u(t, x=L+h)=u(t,x=L)$$ 

 (Centered difference) $$\forall x \in [0,L], \forall t \in [0,T] \\ \partial^2_x u(t,x) \approx \frac{u(t,x+h)+u(t,x-h)-2u(t,x)}{h^2}$$

 This gives us the following discretized problem

 $$ \forall t \in [0,T], \forall j \in \llbracket 1, J \rrbracket \\
    \begin{cases}
    v_0(t)=a \\
    \frac{v_{J+1}(t)-v_J(t)}{h}=0 \\
    v'_j(t)=\beta \frac{v_{j+1}(t)+v_{j-1}-2v_j(t)}{h^2} \\
    v_j(0)=u_1 cos(\frac{\pi x_j}{2L})\\
    \end{cases}$$
    
with 
    $$ (v_j)_{j \in \llbracket 1, J \rrbracket} \in (\R^{\R_+})^{j \in \llbracket 1, J \rrbracket}$$

which can be rewritten in matrix form as 

$$ \begin{cases}
\forall t \in [0,T], V'(t)=\frac{-\beta}{h^2}AV(t)+F \\
V(0)=V_0
\end{cases}$$
with 
$$ V(0)= \begin{bmatrix}
v_1(0) \\
\vdots \\
v_J(0) \end{bmatrix} \\~\\
V(t)= \begin{bmatrix}
v_1(t) \\
\vdots \\~\\
v_J(t) \end{bmatrix} \\~\\
V'(t)= \begin{bmatrix}
v'_1(t) \\
\vdots \\
v'_J(t) \end{bmatrix} \\~\\
A= \begin{bmatrix}
 2 & -1 &  & & 0 \\
 -1 & 2 & -1 &  &  \\
  & \ddots & \ddots & \ddots &  \\
 &  & -1 & 2 & -1\\
0 &  &  & -1 & 1\\ \end{bmatrix} \\~\\
F= \begin{bmatrix}
a \\
0 \\
\vdots \\
0 \end{bmatrix} \\

In [17]:
def matrix_fdm_forward_dnc(J):
    A = 2 * np.eye(J+1)
    for i in range(J+1):
        if i == 0:
            A[i,i+1] = -1
        elif i == J:
            A[i,i-1] = -1
            A[i,i] = 1
        else:
            A[i,i-1] = -1
            A[i,i+1] = -1
    return A

#A = matrix_fdm_forward_dnc(J)


J=10
L=1
a=2
def FDS(L,a,u1, J,b, T):
    h=L/(J+1)
    F= np.append(a, np.zeros(J)).reshape(J+1,1)
    D=((-1)*np.eye(J+1, k=-1)+2*np.eye(J+1)-1*np.eye(J+1,k=1))
    D[J][J]-=1
    V0=np.array([u1*np.cos((np.pi*j*h)/(2*L)) for j in range(1,J+2)])
    print(V0)
    print(F)
    def f(t,X):
       return (-b)/(h**2)*D*X+F
    return solve_ivp(f,(0,T),V0)
FDS(1, 2, 3, 10, 1,10)

[2.96946433e+00 2.87847892e+00 2.72889599e+00 2.52376060e+00
 2.26724872e+00 1.96458220e+00 1.62192245e+00 1.24624504e+00
 8.45197671e-01 4.26944515e-01 8.49830835e-16]
[[2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


ValueError: could not broadcast input array from shape (11,11) into shape (11,)